<a href="https://colab.research.google.com/github/BrunoGeraldine/B3_Prediction_2023/blob/main/B3_ETL_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# B3_Prediction_2023 → ETL_Function

In [1]:
import pyarrow
import pandas as pd

### ETL_Function → read data

In [2]:
def read_files(path, name_file, year_date, type_file):
  _file = f'{path}{name_file}{year_date}.{type_file}'

  colspecs = [(2,10), (10,12), (12,24), (27,39),
              (56,69), (69,82), (82,95), (108,121),
              (152,170), (170,188)
  ]

  names = ['data_pregao', 'cod_bdi', 'sigla_acao', 'nome_acao',
           'preco_abert', 'preco_max', 'preco_min', 'preco_fech',
           'qtd_negocios', 'vol_negocios']

  df = pd.read_fwf(_file, colspecs=colspecs, names=names, skiprows=1, encoding='latin1')

  return df

### Filter_Function

In [3]:


def filter_stocks(df):
  df = df[df['cod_bdi'] == 2] 
  df = df.drop(columns=['cod_bdi'])

  return df

### Date_Function

In [4]:
def parse_date(df):
  df['data_pregao'] = pd.to_datetime(df['data_pregao'], format ='%Y%m%d')
  #df['Year'] = df['data_pregao'].dt.strftime( '%Y')


  return df

### Value_Function

In [5]:
def parse_values(df):
  df['preco_abert'] = (df['preco_abert']  /100).astype(float)
  df['preco_max'] = (df['preco_max']  /100).astype(float)
  df['preco_min'] = (df['preco_min']  /100).astype(float)
  df['preco_fech'] = (df['preco_fech']  /100).astype(float) 

  return df

### Join_Files

In [6]:
def concat_files(path, name_file, year_date, type_file, final_file):
  for i, y in enumerate(year_date):
    df = read_files(path, name_file, y, type_file)
    df = filter_stocks(df)
    df = parse_date(df)
    df = parse_values(df)

    if i==0:
      df_final = df
    else: 
      df_final = pd.concat([df_final, df])

  df_final.to_parquet(f'{path}//{abt_file}', index=False)


### Running_ETL_Function

In [7]:
year_date = ['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2023']

path = f'/home/bruno/repos/B3_Prediction_2023/dataset/'

name_file = 'COTAHIST_A'

type_file = 'TXT'

abt_file = 'all_bovespa_13to23.parquet'

concat_files(path, name_file, year_date, type_file, abt_file)

